In [ ]:
import requests
import pandas as pd
from openai import OpenAI
import dotenv
import os
import re
from tqdm import tqdm

In [ ]:
dotenv.load_dotenv(".envrc")
api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key=api_key)

In [ ]:
with open("resources/question_augment_prompt.txt", "r") as f:
    prompt = f.read()

question_pattern = re.compile(r"\d\. (.*)\n?")

In [ ]:
df = pd.read_csv('input/alternatives.csv')

In [ ]:
correct = 0
total = 0
alternatives = {}
for row in tqdm(df.to_dict(orient="records")):
    question = row["text"]
    content = prompt.format(question=question, n=5)
    conversation = [{"role": "user", "content": content}]
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=conversation, max_tokens=1000)
    text = response.choices[0].message.content

    choices = question_pattern.findall(text)
    alternatives[row["id"]] = choices
    choices = [choice.strip().strip('"') for choice in choices]
    for choice in choices:
        total += 1
        msg = None
        for i in range(3):
            try:
                response = requests.post(url="http://coruscant.local:8000/inference/", json={"question": choice})
            except Exception as e:
                msg = e
                continue
            break
        else:
            print("ERROR", msg)
            continue
        
        result = response.json()
        if not result:
            print(f"Question: {question}")
            print(f"Choice: {choice}")
            print(f"Result: {result}")
            print("-" * 80)
            continue

        first = result[0]["similar_question"]
        if first.strip() == question.strip():
            correct += 1
        else:
            print(f"Question: {question}")
            print(f"Choice: {choice}")
            print(f"Result: {result}")
            print("-" * 80)

In [ ]:
correct, total, correct / total

In [ ]:
rows = []
for key, value in alternatives.items():
    for choice in value:
        rows.append({"id": key, "choice": choice})

df = pd.DataFrame(rows)
df.to_csv("input/alternatives.csv", index=False)